In [1]:
# full tutorial avaiable at:
# https://www.datacamp.com/tutorial/open-ai-function-calling-tutorial

In [ ]:
!pip install openai python-dotenv

In [10]:

import openai
import json
import os
from dotenv import load_dotenv

load_dotenv()


openai.api_key = os.getenv("OPENAI_API_KEY")


## Demo for OpenAI function calling (i.e., the ability to output structred data)

## First, we define a series of functions that we what the LLM to use

In [12]:
custom_functions = [
    {
        'name': 'extract_student_info',
        'description': 'Get the student information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'major': {
                    'type': 'string',
                    'description': 'Major subject.'
                },
                'school': {
                    'type': 'string',
                    'description': 'The university name.'
                },
                'grades': {
                    'type': 'integer',
                    'description': 'GPA of the student.'
                },
                'club': {
                    'type': 'string',
                    'description': 'School club for extracurricular activities. '
                }
                
            }
        }
    },
    {
        'name': 'extract_school_info',
        'description': 'Get the school information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the school.'
                },
                'ranking': {
                    'type': 'integer',
                    'description': 'QS world ranking of the school.'
                },
                'country': {
                    'type': 'string',
                    'description': 'Country of the school.'
                },
                'no_of_students': {
                    'type': 'integer',
                    'description': 'Number of students enrolled in the school.'
                }
            }
        }
    }
]

## Now some potential inputs:

In [29]:
student_1_description = "David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating."
student_2_description="Ravi Patel is a sophomore majoring in computer science at the University of Michigan. He is South Asian Indian American and has a 3.7 GPA. Ravi is an active member of the university's Chess Club and the South Asian Student Association. He hopes to pursue a career in software engineering after graduating."
school_1_description = "Stanford University is a private research university located in Stanford, California, United States. It was founded in 1885 by Leland Stanford and his wife, Jane Stanford, in memory of their only child, Leland Stanford Jr. The university is ranked #5 in the world by QS World University Rankings. It has over 17,000 students, including about 7,600 undergraduates and 9,500 graduates23. "

irrelevant_text = "Tell me about the relationship between power and morality in five words."

In [30]:
#
descriptions = [student_1_description, student_2_description, school_1_description, irrelevant_text]

## Now define some functions for our application that take in structured data from the LLM and output the desired result

In [31]:
def extract_student_info(name, major, school, grades, club):
    
    """Get the student information"""

    return f"{name} is majoring in {major} at {school}. He has {grades} GPA and he is an active member of the university's {club}."

def extract_school_info(name, ranking, country, no_of_students):
    
    """Get the school information"""

    return f"{name} is located in the {country}. The university is ranked #{ranking} in the world with {no_of_students} students."

In [32]:
# Function names
available_functions = {
    "extract_school_info": extract_school_info,
    "extract_student_info": extract_student_info
}

## Now Let's use OpenAI's LLM

In [33]:

"""
Call OpenAI's ChatCompletion API to generate a response for each sample in the descriptions list.
If a function call is detected in the response, it extracts the function name and arguments, and calls the corresponding function.
Otherwise, it returns the response content.
"""
for i, sample in enumerate(descriptions):
    print(f"Sample#{i+1}: \n \"{sample}\" \n")
    print("\tCalling OpenAI API...")
    response = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': sample}],
        functions = custom_functions,
        function_call = 'auto'
    )

    response_message = response["choices"][0]["message"]
    print("\tResponse received!")

    if response_message.get('function_call'):

        print("\tFunction call detected!")

        # Which function call was invoked
        function_called = response_message['function_call']['name']

        # Extracting the arguments
        function_args  = json.loads(response_message['function_call']['arguments'])

        print(f"\tFunction called: {function_called}")
        print(f"\tFunction arguments: {function_args}")


        fuction_to_call = available_functions[function_called]
        response_message = fuction_to_call(*list(function_args .values()))

    else:
        print("\tNo function call detected!")
        response_message = response_message['content']

    print(f"\tResponse: {response_message}")

    print("--------------------------------------")


Sample#1: 
 "David Nguyen is a sophomore majoring in computer science at Stanford University. He is Asian American and has a 3.8 GPA. David is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after graduating." 

	Calling OpenAI API...
	Response received!
	Function call detected!
	Function called: extract_student_info
	Function arguments: {'name': 'David Nguyen', 'major': 'computer science', 'school': 'Stanford University', 'grades': 3.8, 'club': 'Robotics Club'}
	Response: David Nguyen is majoring in computer science at Stanford University. He has 3.8 GPA and he is an active member of the university's Robotics Club.
--------------------------------------
Sample#2: 
 "Ravi Patel is a sophomore majoring in computer science at the University of Michigan. He is South Asian Indian American and has a 3.7 GPA. Ravi is an active member of the university's Chess Club and the South Asian Student As